# morphofit example usage notebook

In this notebook, we provide examples on how to run morphofit on a set of two simulated HST images mimicking the MACS J1149.5+2223 observational conditions in the Frontier Fields survey in the F814W and F160W wavebands.

morphofit is designed to be ran via command-line using esub-epipe. It can also be run in a notebook either by calling the scripts in the `main_functions` folder with `subprocess` or by calling the individual functions or the main function of the scripts in the notebook cells.

In order to run it, the user should create a folder named as the `target_name`, e.g. macs1149. All initial and produced data are going to be stored in this folder. The user should also rename images following the naming convention:

`telescopename_targetname_waveband_imagetype.fits`

e.g. HST_macs1149_f814_drz.fit

See the folder `demo_data` for an example.

__Example of calling the morphofit modules in the `main_functions` folder with the syntax in the `pipeline.yaml` file__

__How to use morphofit to run SExtractor in forced photometry mode on all provided images in all wavebands__

In [ ]:
import subprocess
import os

mode = 'run-tasks' # whether to run the script in serial (run) or in parallel (run-tasks, run-mpi)
tasks = '0>1' # total number of jobs to run, e.g. '0>3' is the syntax for running SExtractor on
              # three different galaxy clusters. Since we have one cluster example, tasks is just 0>1
              # in the case of galfit fit on stamps, the number of tasks is given by 
              # n_target_galaxies * n_bands * n_combinations_background_sigmaimage_psf
root_path = './demo_data/'# Path to root folder where data are stored and where code is run
wavebands_list = 'f814w,f160w' # string of comma-separated waveband names
h5pytable_folder = './demo_data/h5table/' # path to folder where the hdf5 table with parameters relevant for 
                                          # morphofit are stored
h5pytable_prefix = 'sextractor_run_table' # filename prefix of hdf5 table storing parameters relevant for morphofit 
telescope_name = 'HST' # name of the telescope/instrument/survey data used
target_field_names = 'macs1149' # name of the target to analyse, it has to be the same as targetname
sci_images_suffix = 'drz.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the science image
rms_images_suffix = 'rms.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the root mean square image, if unavailable set it to 'None'
exp_images_suffix = 'exp.fits' # it has to be the same as imagetype.fits and represent the suffix with which 
                               # we define the exposure time image, if unavailable set it to 'None'
ext_star_cat_suffix = 'star_positions_f160w.fits' # suffix of the fits table storing the star positions in the image
image_archive_prefix = 'images' # filename prefix of the generated tar file containing images
resources_archive_prefix = 'res_sextractor_files' # filename prefix of the generated tar file containing res files
star_catalogues_path = './demo_data/star_catalogues/'# path to star catalogues folder
pixel_scale = 0.060 # pixel scale in arcsec/pixel
seeing_initial_guesses = '0.1,0.1' # initial guesses for the seeing FWHM in arcsec, e.g. 0.1 arcsec for HST
sextractor_binary_filename = '/opt/homebrew/bin/sex' # path to SExtractor executable
# from detect_minarea to sextractor_checkimages_endings, the parameters have the same meaning
# of the corresponding SExtractor ones
sextractor_resources_path = '../morphofit/res/sextractor/' # path to folder storing SExtractor required files
function = 'all' # it can be all, main, check_missing and merge
n_cores = os.cpu_count() # number of available CPU for parallel computing

subprocess.run(['esub', '../main_functions/create_table_for_sextractor.py', '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--wavebands_list={}'.format(wavebands_list),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--telescope_name={}'.format(telescope_name),
                '--target_field_names={}'.format(target_field_names),
                '--sci_images_suffix={}'.format(sci_images_suffix),
                '--rms_images_suffix={}'.format(rms_images_suffix),
                '--exp_images_suffix={}'.format(exp_images_suffix),
                '--ext_star_cat_suffix={}'.format(ext_star_cat_suffix),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--star_catalogues_path={}'.format(star_catalogues_path),
                '--pixel_scale={}'.format(pixel_scale), '--seeing_initial_guesses={}'.format(seeing_initial_guesses),
                '--detect_minarea=10', '--detect_thresh=1.0', '--analysis_thresh=1.5', '--deblend_nthresh=64',
                '--deblend_mincont=0.0001', '--phot_apertures=3,5,10,15,20,25',
                '--phot_autoparams=2.5,3.5', '--phot_petroparams=2.0,3.5', '--phot_autoapers=0.0,0.0',
                '--phot_fluxfrac=0.5', '--back_size=64', '--back_filtersize=3',
                '--sextractor_binary_filename={}'.format(sextractor_binary_filename),
                '--sextractor_config_filename=default.sex', '--sextractor_params_filename=default.param',
                '--sextractor_filter_filename=gauss_3.0_5x5.conv', '--sextractor_nnw_filename=default.nnw',
                '--sextractor_checkimages=SEGMENTATION', '--sextractor_checkimages_endings=seg',
                '--sextractor_resources_path={}'.format(sextractor_resources_path),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])


In [ ]:
temp_dir_path = './demo_data/' # temporary folders location where morphofit single core jobs are run
local_or_cluster = 'local' # it can be either local or cluster, set it to local
sextractor_forced_catalogue_suffix = 'forced.cat' # user-defined suffix for the single-band output SExtractor catalogue
sextractor_ra_keyword = 'ALPHAWIN_J2000' # right ascension column name to be used to match galaxies across bands
sextractor_dec_keyword = 'DELTAWIN_J2000' # declination column name to be used to match galaxies across bands
star_catalogue_ra_keyword = 'ra' # right ascension column name to be used to find stars in images and measure 
                                 # their profile FWHM
star_catalogue_dec_keyword = 'dec' # declination column name to be used to find stars in images and measure 
                                   # their profile FWHM
detection_image_suffix = 'detection.fits' # suffix of the generated detection image
parameters_table_suffix = 'param_table.fits' # suffix of the fits table storing obtained instrumental and 
                                             # observational parameters
multiband_catalogue_suffix = 'multiband.forced.cat' # user-defined suffix for the multiband output SExtractor catalogue

subprocess.run(['esub', '../main_functions/main_sextractor.py', '--mode={}'.format(mode), 
                '--tasks={}'.format(tasks), '--root_path={}'.format(root_path),
                '--h5pytable_folder={}'.format(h5pytable_folder),
                '--h5pytable_prefix={}'.format(h5pytable_prefix),
                '--image_archive_prefix={}'.format(image_archive_prefix),
                '--resources_archive_prefix={}'.format(resources_archive_prefix),
                '--temp_dir_path={}'.format(temp_dir_path),
                '--local_or_cluster={}'.format(local_or_cluster),
                '--sextractor_forced_catalogue_suffix={}'.format(sextractor_forced_catalogue_suffix),
                '--sextractor_checkimages_endings=seg',
                '--sextractor_ra_keyword={}'.format(sextractor_ra_keyword),
                '--sextractor_dec_keyword={}'.format(sextractor_dec_keyword),
                '--star_catalogue_ra_keyword={}'.format(star_catalogue_ra_keyword),
                '--star_catalogue_dec_keyword={}'.format(star_catalogue_dec_keyword),
                '--detection_image_suffix={}'.format(detection_image_suffix),
                '--parameters_table_suffix={}'.format(parameters_table_suffix),
                '--multiband_catalogue_suffix={}'.format(multiband_catalogue_suffix),
                '--function={}'.format(function), '--n_cores={}'.format(n_cores)
                ])